<a href="https://colab.research.google.com/github/Tbot101/CIS-5190-Project/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/CIS\ 519\ Final\ Project

/content/drive/Shareddrives/CIS 519 Final Project


In [ ]:
%ls
# 'Reviews.csv' should be listed here

'CIS 519 Project.ipynb'       'Project Milestone 2.gdoc'
 cleaned_reviews.csv          'Project Milestone 3.docx'
 fine-tuned-bert               res/
 James_CIS_519_Project.ipynb   Reviews.csv
 logs4/                       'Taha CIS 519 Project'
'Project 1 Milestone.gdoc'    'Time-Shift CIS 519 Project'


In [ ]:
import pandas as pd
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

random.seed(42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
USE_CLEANED = True

In [ ]:
# we need to use these parameters to read in some funky text
if USE_CLEANED:
  df = pd.read_csv('cleaned_reviews.csv', engine='python', on_bad_lines='skip')
else:
  df = pd.read_csv('Reviews.csv', engine='python', on_bad_lines='skip')
  df['sentiment'] = df.apply(lambda x: 1 if x['Score'] > 3 else 0, axis=1)
  df = df.sample(frac=0.2).reset_index(drop=True)
  def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    lemma = WordNetLemmatizer()
    result = [i for i in tokens if not i in stop_words]
    lemmatized_result = []
    for word in result:
      lemmatized_result.append(lemma.lemmatize(word))

    return " ".join(lemmatized_result)
  df['cleaned_text'] = df.apply(lambda x: preprocess_text(x['Text']), axis=1)

In [ ]:
df = df.dropna()

In [ ]:
df

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment,cleaned_text
0,0,119814,B0009YUEG2,A2F09EWKV3MTO2,StillWaters,4,4,5,1303430400,This is all I ever use anymore,I love this litter because...<br />1. It compl...,1,love litter br 1 completely eliminates urine o...
1,1,290509,B004777F3M,A2PDNBEIQYRCT2,lmac60,0,0,4,1309996800,Good Tea!,Great flavor. Just enough spice. A love Chai...,1,great flavor enough spice love chai tea latte ...
2,2,489194,B002N2R7WM,A1CHGDG3NUDA5R,"C. Seggelin ""Plastered Dragon""",0,0,1,1350086400,Tasteless Fat For Your Popcorn! Just Use Butter.,"Y'know, you'd think with the words ""Buttery Fl...",0,know think word buttery flavor product name pr...
3,3,65205,B001D0GV5O,A3GIKK6DQEKEQK,CHERYL A. CATANZANO,0,0,1,1296777600,Very bland,I bought this tea because I like Timothy's Cof...,0,bought tea like timothy coffee product figured...
4,4,526585,B007PA30TG,A2XQSJG8YR6ED2,Cecilia f.,0,0,5,1303084800,great brew,i've been using these k-cups for months now. ...,1,using k cup month breakfast blend perfect peop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113686,113686,16241,B007TJGZ54,A1PLASEQUSPDPM,Chris,0,0,5,1339718400,Price is Right,Love the Green Mountain Coffee; tastes great i...,1,love green mountain coffee taste great keurig ...
113687,113687,511398,B001E6J07I,A233H2J16V56W8,"Claxon,Bill",0,0,5,1237507200,Tasty Carrs cracker,A super glamourized cheez-it without the trans...,1,super glamourized cheez without trans fat chol...
113688,113688,211139,B000X67P2C,A3G0H0GKJ0722I,C. Smith,10,13,1,1277164800,Did not like...,I had heard so many good things about using th...,0,heard many good thing using place wheat white ...
113689,113689,520860,B0009XSXZM,A3FTHCRY3EZNA7,categ3893,1,1,5,1333843200,8 month Golden Retriever is OBSESSED.,My adolescent golden retriever just loves thes...,1,adolescent golden retriever love soon see hold...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df)

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
baseline = DummyClassifier(strategy="most_frequent")

In [ ]:
baseline.fit(train.drop('sentiment', axis=1), train['sentiment'])

DummyClassifier(strategy='most_frequent')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, log_loss



In [ ]:
baseline_predictions = baseline.predict(test.drop('sentiment', axis=1))
baseline_accuracy = accuracy_score(test['sentiment'], baseline_predictions)
baseline_accuracy

0.7792751583391977

In [ ]:
baseline_f1score = f1_score(test['sentiment'], baseline_predictions)
baseline_f1score

0.875946763699646

In [ ]:
def get_metrics(predictions, actual):
  accuracy = accuracy_score(actual, predictions)
  f1 = f1_score(actual, predictions)
  cross_entropy = log_loss(actual, predictions)
  print(f"Accuracy: {accuracy:.2f}")
  print(f"F1 Score: {f1:.2f}")
  print(f"Cross-Entropy: {cross_entropy:.2f}")

In [ ]:
print("-- Baseline metrics --")
get_metrics(baseline_predictions, test['sentiment'])

-- Baseline metrics --
Accuracy: 0.78
F1 Score: 0.88
Cross-Entropy: 7.96


In [ ]:
def shift_negative_data(test_data):
  negative_data = test_data[test_data['sentiment'] == 0]
  negative_count = len(negative_data)
  positive_data = test_data[test_data['sentiment'] == 1]
  positive_sample = positive_data.sample(n=negative_count)
  return positive_sample.append(negative_data)

In [ ]:
print("Original data:")
print(test['sentiment'].describe())
print("Shifted data:")
shifted_negative_test = shift_negative_data(test)
print(shifted_negative_test['sentiment'].describe())

Original data:
count    28420.000000
mean         0.779275
std          0.414743
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: sentiment, dtype: float64
Shifted data:
count    12546.00000
mean         0.50000
std          0.50002
min          0.00000
25%          0.00000
50%          0.50000
75%          1.00000
max          1.00000
Name: sentiment, dtype: float64


<ipython-input-19-4a862ef7d6af>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return positive_sample.append(negative_data)


In [ ]:
baseline_predictions_shifted = baseline.predict(shifted_negative_test.drop('sentiment', axis=1))
print("-- Baseline metrics --")
get_metrics(baseline_predictions_shifted, shifted_negative_test['sentiment'])

-- Baseline metrics --
Accuracy: 0.50
F1 Score: 0.67
Cross-Entropy: 18.02


In [ ]:
# https://towardsdatascience.com/the-secret-to-improved-nlp-an-in-depth-look-at-the-nn-embedding-layer-in-pytorch-6e901e193e16
# ^ Embedding text